# NYC TLC Data Analytics

This goal of the project is to perform data analysis on New York City Taxi and Limousine Commission dataset which is similar to dataset of ridesharing companies such as Uber, OLA, DiDi, etc.
The dataset used can be found here: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page, for this project we're using Janurary 2023 Yellow Taxi Trip Records.

In [ ]:
import pyarrow.parquet as pq
import pandas as pd

In [ ]:
trips = pq.read_table('data/yellow_tripdata_2023-01.parquet')

In [ ]:
df = trips.to_pandas()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates().dropna().reset_index(drop=True)

In [ ]:
df['trip_id'] = df.index

### Datetime Dimension Table

In [ ]:
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['pickup_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pickup_date'] = datetime_dim['tpep_pickup_datetime'].dt.date
datetime_dim['pickup_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_date'] = datetime_dim['tpep_dropoff_datetime'].dt.date
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

datetime_dim['datetime_id'] = datetime_dim.index

In [ ]:
# Order the columns
datetime_dim = datetime_dim[['datetime_id','tpep_pickup_datetime', 'pickup_hour', 'pickup_date', 'pickup_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_date', 'drop_weekday']]

In [ ]:
datetime_dim.to_dict()

### Passenger Count & Trip Distance Dimension Table

In [ ]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]

trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]

### Ratecode Dimension Table

In [ ]:
rate_code_type = {
    1: "Standard Rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated Fare",
    6: "Group Ride"
}

rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id', 'RatecodeID', 'rate_code_name']]

### Pickup & Dropoff Location Dimension Tables

In [ ]:
pickup_location_dim = df[['PULocationID']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','PULocationID']] 


dropoff_location_dim = df[['DOLocationID']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','DOLocationID']]

### Payment Type Dimension Table

In [ ]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}

payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim['payment_type_id'] = payment_type_dim.index

payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_name']]

### Fact Table

In [ ]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

In [ ]:
fact_table